In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from bertopic.representation import KeyBERTInspired
import numpy as np

import re
import nltk
from nltk.corpus import stopwords


In [ ]:
# !pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('path_to_your_folder')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
#df = pd.read_csv("/kaggle/input/sample-reviews/reviews.csv")
df=pd.read_csv('movie_review.csv')
print(df.shape, df['labels'].value_counts())

(48297, 3) labels
positif    25000
négatif    23297
Name: count, dtype: int64


In [ ]:
df.head()

,movie,text,labels
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif


**Traitement du text**

In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = ' '.join(word for word in text.split() if word not in stop_words) #remove stopwords
    return text
df["text_clean"] = df["text"].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,movie,text,labels,text_clean
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,funny thrill ride movie doesnt stop last time ...
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,real good miniseries saw miniseries got hooked...
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,fun movie movie doesnt need lots hitech specia...
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,good film something lacking chemistry good fil...
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,great drama movie philadelphia one best movies...


**Construction du corpus**

In [ ]:
# emotion_words = [
#     "joy", "happy", "cheerful", "delighted", "hopeful", "bliss", "smile", "heartwarming",
#     "sad", "sorrow", "heartbroken", "down", "gloomy", "tearful", "touching", "moving",
#     "angry", "furious", "irritated", "outraged", "hate", "annoyed", "aggression",
#     "afraid", "scared", "terrified", "anxious", "horror", "unsettling", "haunting",
#     "surprised", "shocked", "unexpected", "amazed", "twist", "unbelievable", "wow",
#     "disgusted", "gross", "nasty", "revolting", "cringe", "filthy", "repulsive"
# ]
# emotion_words2 = ["joy", "sad", "angry", "afraid", "surprised", "disguted"]


**Topic Modeling**

In [ ]:
docs = df["text_clean"].tolist()
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
representation_model = KeyBERTInspired()
vectorizer_model = CountVectorizer(stop_words="english")

topic_model = BERTopic(language="english",
                       nr_topics=10,
                       embedding_model=embedding_model,
                       representation_model = representation_model,
                       vectorizer_model=vectorizer_model,)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
df["topic"] = topics
#df["topic_prob"] = probs

In [ ]:
df.head()

,movie,text,labels,text_clean,topic
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,funny thrill ride movie doesnt stop last time ...,-1
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,real good miniseries saw miniseries got hooked...,-1
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,fun movie movie doesnt need lots hitech specia...,-1
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,good film something lacking chemistry good fil...,-1
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,great drama movie philadelphia one best movies...,-1


In [ ]:
topic_model.visualize_topics(top_n_topics=10).show()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10).show()

In [ ]:
# define the theme
topic_to_theme = {
   -1: "other",
    0: "General Cinema",
    1: "Classic Comedy and TV Shows",
    2: "Christmas and Holiday Stories",
    3: "Wrestling and Sports Entertainment",
    4: "South American Culture",
    5: "Cartoons and Kids Characters",
    6: "Fast Food and Health",
    7: "TV Shows and Action Series",
    8: "Home Media and Miniseries"
}

In [ ]:
df["theme"] = df["topic"].map(topic_to_theme)

In [ ]:
df.head(20)

,movie,text,labels,text_clean,topic,theme
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,funny thrill ride movie doesnt stop last time ...,-1,other
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,real good miniseries saw miniseries got hooked...,-1,other
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,fun movie movie doesnt need lots hitech specia...,-1,other
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,good film something lacking chemistry good fil...,-1,other
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,great drama movie philadelphia one best movies...,-1,other
5,Maciste e la regina di Samar (1964),The eternal sandstorm! I guess that would make...,négatif,eternal sandstorm guess would make gritty movi...,-1,other
6,The Monster Walks (1932),"Low...lower...lowest budget ""The Monster Walks...",négatif,lowlowerlowest budget monster walks old cheap ...,-1,other
7,Surplus: Terrorized Into Being Consumers (2003),Cluttered and confused This film works despera...,négatif,cluttered confused film works desperately atta...,-1,other
8,The Hot Chick (2002),Schneider As a Sight Gag I have to hand it to ...,négatif,schneider sight gag hand rob schneider ripping...,-1,other
9,Dead of Night (1945),It's Deja Vu all over again This is one of tho...,positif,deja vu one horror stories diminish years arch...,-1,other


In [ ]:
df.iloc[10]

,10
movie,Night Watch (2004)
text,"The Matrix's Geekier, Russian Cousin The trail..."
labels,négatif
text_clean,matrixs geekier russian cousin trailer film ma...
topic,0
theme,General Cinema


In [ ]:
df["text_clean"].iloc[10]

'matrixs geekier russian cousin trailer film made night watch nochnoy dozor russians say look word stunning cgi impressive outstanding visuals roared background searing rockmusic story looked interesting unusual trailer radiated cool rushed along cinema watch opening night didnt care russian film looked good could deal subtitles oh wrong film least would made sense unless id scrutinised screen read quicker light whilst watching actors screen awful lot thinking hour head hurting eyes becoming lazy night watch story group goodies battling group baddies desperately trying save world evil forces bad men day watch possess main character anton played well konstantin khabebsky ordinary guy decides beginning film wants kill cheating wifes unborn baby anyone else would situation asks crazy old witch group old raven invested flats im sure mama would told go near beginning scene long long theres awful lot understand end night watch arrive realise anton special powers almost expected trinity morph

In [ ]:
# df.to_csv("movie_with_theme", index=False)

**Extract the emotion**

In [ ]:
from transformers import pipeline

emotion_model = pipeline("text-classification",
                         model="j-hartmann/emotion-english-distilroberta-base",
                         truncation=True ,
                         return_all_scores=False)

pred =emotion_model(df["text_clean"].iloc[10])
print( pred, pred[0]['label'])

Device set to use cuda:0


[{'label': 'sadness', 'score': 0.581184446811676}] sadness


In [ ]:
df["emotion"] = df["text_clean"].apply(lambda x: emotion_model(x)[0]['label'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df.head(30)

,movie,text,labels,text_clean,topic,theme,emotion
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,funny thrill ride movie doesnt stop last time ...,-1,other,joy
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,real good miniseries saw miniseries got hooked...,-1,other,fear
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,fun movie movie doesnt need lots hitech specia...,-1,other,joy
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,good film something lacking chemistry good fil...,-1,other,joy
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,great drama movie philadelphia one best movies...,-1,other,sadness
5,Maciste e la regina di Samar (1964),The eternal sandstorm! I guess that would make...,négatif,eternal sandstorm guess would make gritty movi...,-1,other,anger
6,The Monster Walks (1932),"Low...lower...lowest budget ""The Monster Walks...",négatif,lowlowerlowest budget monster walks old cheap ...,-1,other,fear
7,Surplus: Terrorized Into Being Consumers (2003),Cluttered and confused This film works despera...,négatif,cluttered confused film works desperately atta...,-1,other,anger
8,The Hot Chick (2002),Schneider As a Sight Gag I have to hand it to ...,négatif,schneider sight gag hand rob schneider ripping...,-1,other,joy
9,Dead of Night (1945),It's Deja Vu all over again This is one of tho...,positif,deja vu one horror stories diminish years arch...,-1,other,fear


In [ ]:
data = df.copy()
data.drop(["text_clean", "topic"],axis=1,inplace=True)
data.to_csv("movie_with_theme_and_emotion.csv", index=False)
data.head()

,movie,text,labels,theme,emotion
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,other,joy
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,other,fear
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,other,joy
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,other,joy
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,other,sadness


In [ ]:
df=pd.read_csv('movie_with_theme_and_emotion.csv')
df.head(30)

,movie,text,labels,theme,emotion
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif,other,joy
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif,other,fear
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif,other,joy
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif,other,joy
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif,other,sadness
5,Maciste e la regina di Samar (1964),The eternal sandstorm! I guess that would make...,négatif,other,anger
6,The Monster Walks (1932),"Low...lower...lowest budget ""The Monster Walks...",négatif,other,fear
7,Surplus: Terrorized Into Being Consumers (2003),Cluttered and confused This film works despera...,négatif,other,anger
8,The Hot Chick (2002),Schneider As a Sight Gag I have to hand it to ...,négatif,other,joy
9,Dead of Night (1945),It's Deja Vu all over again This is one of tho...,positif,other,fear


In [ ]:
grouped_df = df.groupby(["movie", "labels", "emotion"]).size().reset_index(name="count")
grouped_df.head(30)

,movie,labels,emotion,count
0,$ (1971),positif,joy,1
1,'15' (2002),négatif,anger,1
2,"'Ctyri vrazdy stací, drahousku' (1971)",positif,surprise,1
3,'D' (2005),positif,joy,1
4,'Doc' (1971),négatif,surprise,1
5,'Halloween' Unmasked 2000 (1999 Video),positif,joy,1
6,'It's Alive!' (1969 TV Movie),positif,fear,1
7,'It's Alive!' (1969 TV Movie),positif,joy,1
8,'N Sync: PopOdyssey Live (2002 TV Special),positif,joy,1
9,'N Sync: The Atlantis Concert (2001 TV Special),positif,surprise,1


In [ ]:
#Pour l'extraction des sentiments de l'utilisateur
from transformers import pipeline

emotion_model = pipeline("text-classification",
                         model="j-hartmann/emotion-english-distilroberta-base",
                         truncation=True ,
                         return_all_scores=False)

user_entry= "I am happy"  ## le texte que l'utilisateur devra écricre
pred =emotion_model(user_entry)
user_emotion = pred[0]['label']
print( user_emotion)   ## user_emotion = le champ de filtre à comparer avec le champ emotion dans la base de données (select films where user_emotion = emotion)

joy
